In [7]:
import sys
sys.path.append('../')

import pyterrier as pt
import pandas as pd
import os
from src.create_index import load_data, load_folds

In [3]:
RESULTS_PATH = "../data/results"

In [20]:
results = pd.DataFrame()
sub_collections = [
    "t1",
    "t2",
    "t3",
    "t4",
    "t5",
]

folds = load_folds()

run_files = os.listdir(RESULTS_PATH)
for sub_collection in sub_collections:
    
    topics, qrels = load_data(sub_collection)
    
    for fold_no in range(0, len(folds[sub_collection])):
        fold_no = str(fold_no)
        train_docids = folds[sub_collection][fold_no]["train"]
        test_docids = folds[sub_collection][fold_no]["test"]

        runs = []
        run_names = []
        for run_file in run_files:
            if sub_collection in run_file and "F"+fold_no in run_file:
                run_name = run_file
                if "extended" in run_name or "long" in run_name:
                    continue
                
                run_names.append(run_name)
                
                run = pt.io.read_results(os.path.join(RESULTS_PATH, run_name))
                
                runs.append(run)
        
        
        qrels = qrels[~qrels["docno"].isin(train_docids)]
        res = pt.Experiment(
        runs,
        topics,
        qrels,
        names=run_names,
        eval_metrics=["ndcg", "bpref", "map", "ndcg_cut.10", "P.10", "recall.100"],
        verbose=True
        ) 
        
        res["sub_collection"] = sub_collection
        res["fold_no"] = fold_no
        results = pd.concat([results, res])
    

In [22]:
results

,name,ndcg,bpref,map,ndcg_cut.10,P.10,recall.100,sub_collection,fold_no
0,BM25_t1_F0,0.197735,0.216435,0.107220,0.127888,0.063152,0.353313,t1,0
1,BM25+qrel_boost_t1_F0-_l-0.5_m-2,0.197735,0.216435,0.107220,0.127888,0.063152,0.353313,t1,0
2,BM25+RF_t1_F0-t0,0.196992,0.216151,0.106391,0.126907,0.062925,0.353313,t1,0
0,BM25+qrel_boost_t1_F2-_l-0.5_m-2,0.202389,0.223423,0.109605,0.131112,0.063265,0.366588,t1,2
1,BM25+RF_t1_F2-t0,0.201838,0.222289,0.109225,0.130346,0.063039,0.364320,t1,2
